In [ ]:
%sh
wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract10.xml.gz
wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract11.xml.gz
wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract12.xml.gz
wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract13.xml.gz
wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract14.xml.gz
wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract15.xml.gz
wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract16.xml.gz
wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract17.xml.gz

--2022-11-30 08:19:13--  https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-abstract10.xml.gz
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28782197 (27M) [application/octet-stream]
Saving to: ‘enwiki-latest-abstract10.xml.gz.3’

     0K .......... .......... .......... .......... ..........  0%  686K 41s
    50K .......... .......... .......... .......... ..........  0%  686K 41s
   100K .......... .......... .......... .......... ..........  0%  685K 41s
   150K .......... .......... .......... .......... ..........  0%  118M 31s
   200K .......... .......... .......... .......... ..........  0%  691K 32s
   250K .......... .......... .......... .......... ..........  1% 69.8M 27s
   300K .......... .......... .......... .......... ..........  1% 95.2M 23s
   350K .......... .........

In [ ]:
%sh 
ls


azure
conf
enwiki-latest-abstract10.xml.gz
enwiki-latest-abstract10.xml.gz.1
enwiki-latest-abstract10.xml.gz.2
enwiki-latest-abstract10.xml.gz.3
enwiki-latest-abstract11.xml.gz
enwiki-latest-abstract11.xml.gz.1
enwiki-latest-abstract11.xml.gz.2
enwiki-latest-abstract11.xml.gz.3
enwiki-latest-abstract12.xml.gz
enwiki-latest-abstract12.xml.gz.1
enwiki-latest-abstract13.xml.gz
enwiki-latest-abstract13.xml.gz.1
enwiki-latest-abstract14.xml.gz
enwiki-latest-abstract14.xml.gz.1
enwiki-latest-abstract15.xml.gz
enwiki-latest-abstract15.xml.gz.1
enwiki-latest-abstract16.xml.gz
enwiki-latest-abstract16.xml.gz.1
enwiki-latest-abstract17.xml.gz
enwiki-latest-abstract17.xml.gz.1
eventlogs
ganglia
hadoop_accessed_config.lst
logs
metastore_db
preload_class.lst


In [ ]:
# dbutils.shell("cp enwiki-latest-abstract10.xml.gz dbfs://enwiki-latest-abstract10.xml.gz")
# dbutils.FSHandler.cp("enwiki-latest-abstract10.xml.gz", "dbfs://enwiki-latest-abstract10.xml.gz")
dbutils.fs.cp("file:///databricks/driver/enwiki-latest-abstract10.xml.gz", "dbfs:/enwiki-latest-abstract10.xml.gz")
dbutils.fs.cp("file:///databricks/driver/enwiki-latest-abstract11.xml.gz", "dbfs:/enwiki-latest-abstract11.xml.gz")
dbutils.fs.cp("file:///databricks/driver/enwiki-latest-abstract12.xml.gz", "dbfs:/enwiki-latest-abstract12.xml.gz")
dbutils.fs.cp("file:///databricks/driver/enwiki-latest-abstract13.xml.gz", "dbfs:/enwiki-latest-abstract13.xml.gz")
dbutils.fs.cp("file:///databricks/driver/enwiki-latest-abstract14.xml.gz", "dbfs:/enwiki-latest-abstract14.xml.gz")
dbutils.fs.cp("file:///databricks/driver/enwiki-latest-abstract15.xml.gz", "dbfs:/enwiki-latest-abstract15.xml.gz")
dbutils.fs.cp("file:///databricks/driver/enwiki-latest-abstract16.xml.gz", "dbfs:/enwiki-latest-abstract16.xml.gz")
dbutils.fs.cp("file:///databricks/driver/enwiki-latest-abstract17.xml.gz", "dbfs:/enwiki-latest-abstract17.xml.gz")

Out[31]: True

In [ ]:
fnames = ["enwiki-latest-abstract10.xml.gz","enwiki-latest-abstract11.xml.gz","enwiki-latest-abstract12.xml.gz","enwiki-latest-abstract13.xml.gz","enwiki-latest-abstract14.xml.gz","enwiki-latest-abstract15.xml.gz","enwiki-latest-abstract16.xml.gz","enwiki-latest-abstract17.xml.gz"]
df = spark.read.format("xml").option("rootTag", "feed").option("rowTag", "doc").load(','.join(fnames))

In [ ]:
df.printSchema()

root
 |-- abstract: string (nullable = true)
 |-- links: struct (nullable = true)
 |    |-- sublink: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _linktype: string (nullable = true)
 |    |    |    |-- anchor: string (nullable = true)
 |    |    |    |-- link: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)



In [ ]:
from pyspark.sql.functions import monotonically_increasing_id 
indexed_df = df.withColumn("id", monotonically_increasing_id())
indexed_df.show(5)

+--------------------+--------------------+--------------------+--------------------+---+
|            abstract|               links|               title|                 url| id|
+--------------------+--------------------+--------------------+--------------------+---+
|| locmapin     = ...|{[{nav, Buildings...|Wikipedia: Giffor...|https://en.wikipe...|  0|
|Walter Hoppe (Mar...|{[{nav, Literatur...|Wikipedia: Walter...|https://en.wikipe...|  1|
|Cymbalophora oert...|{[{nav, Reference...|Wikipedia: Cymbal...|https://en.wikipe...|  2|
|        Graham McRae|{[{nav, Races, ht...|Wikipedia: 1973 T...|https://en.wikipe...|  3|
|| broke_ground   ...|{[{nav, External ...|Wikipedia: Pors S...|https://en.wikipe...|  4|
+--------------------+--------------------+--------------------+--------------------+---+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import split, col,lower
lower_df = indexed_df.select("*", lower(col('abstract')).alias('lower_abstract'))
tokenized_df = lower_df.select('abstract','title','url','id','lower_abstract')
tokenized_df.show(5)

+--------------------+--------------------+--------------------+---+--------------------+
|            abstract|               title|                 url| id|      lower_abstract|
+--------------------+--------------------+--------------------+---+--------------------+
|| locmapin     = ...|Wikipedia: Giffor...|https://en.wikipe...|  0|| locmapin     = ...|
|Walter Hoppe (Mar...|Wikipedia: Walter...|https://en.wikipe...|  1|walter hoppe (mar...|
|Cymbalophora oert...|Wikipedia: Cymbal...|https://en.wikipe...|  2|cymbalophora oert...|
|        Graham McRae|Wikipedia: 1973 T...|https://en.wikipe...|  3|        graham mcrae|
|| broke_ground   ...|Wikipedia: Pors S...|https://en.wikipe...|  4|| broke_ground   ...|
+--------------------+--------------------+--------------------+---+--------------------+
only showing top 5 rows



In [ ]:
query = [['abstract','title','url','id'], ['The birth place is Belgium','Birth place','https://en.wikipedia.org/wiki/1973_Tasman_Series','20909090']]
queryDF = spark.createDataFrame(query[1:],query[0])
qlower_df = queryDF.select("*", lower(col('abstract')).alias('lower_abstract'))
Qtoken_df = qlower_df
Qtoken_df.show()

+--------------------+-----------+--------------------+--------+--------------------+
|            abstract|      title|                 url|      id|      lower_abstract|
+--------------------+-----------+--------------------+--------+--------------------+
|The birth place i...|Birth place|https://en.wikipe...|20909090|the birth place i...|
+--------------------+-----------+--------------------+--------+--------------------+



In [ ]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml import Pipeline
inCol = 'abstract'
# def evaluate(inCol):
minDFs = {'abstract':2.0}
preProcStages = []
regexTokenizer = RegexTokenizer(gaps=False, pattern='\w+', inputCol=inCol, outputCol=inCol+'Token')
stopWordsRemover = StopWordsRemover(inputCol=inCol+'Token', outputCol=inCol+'SWRemoved')
countVectorizer = CountVectorizer(minDF=minDFs['abstract'], inputCol=inCol+'SWRemoved', outputCol=inCol+'TF')
idf = IDF(inputCol=inCol+'TF', outputCol=inCol+'IDF') 
preProcStages += [regexTokenizer, stopWordsRemover, countVectorizer, idf]

pipeline = Pipeline(stages=preProcStages)

In [ ]:
dataCombined = tokenized_df.union(Qtoken_df)
model = pipeline.fit(dataCombined)
# model.transform(tokenized_df).show(5)
tf_idf_model = model.transform(dataCombined)

In [ ]:
from pyspark.sql.functions import count
queryCombined = tf_idf_model.select('abstract', 'title', 'id', 'url', 'abstractTF', 'abstractIDF','abstractSWRemoved')
# queryCombined = tf_idf_model.select('abstract', 'title', 'id', 'url')
queryCombined.sample(False, .01).show(5)
queryCombined.select(count(queryCombined.id)).show()

+--------------------+--------------------+---+--------------------+--------------------+--------------------+--------------------+
|            abstract|               title| id|                 url|          abstractTF|         abstractIDF|   abstractSWRemoved|
+--------------------+--------------------+---+--------------------+--------------------+--------------------+--------------------+
|      | recorded   =|Wikipedia: Fallin...| 11|https://en.wikipe...| (262144,[47],[1.0])|(262144,[47],[4.2...|          [recorded]|
|In Pakistan, a te...|Wikipedia: List o...|504|https://en.wikipe...|(262144,[31,91,17...|(262144,[31,91,17...|[pakistan, tehsil...|
|           | image =|Wikipedia: Helcio...|657|https://en.wikipe...|(262144,[149],[1.0])|(262144,[149],[4....|             [image]|
|The men's individ...|Wikipedia: Gymnas...|667|https://en.wikipe...|(262144,[19,52,56...|(262144,[19,52,56...|[men, individual,...|
|The Brew Company ...|Wikipedia: The Br...|777|https://en.wikipe...|(262144,

In [ ]:
import math
def cosine_similarity(X, Y):
  denom = X.norm(2) * Y.norm(2)
  if denom == 0.0:
    return -1.0
  else:
    return X.dot(Y) / float(denom)

In [ ]:
docID1= 6
docID2 =20909090
doc1 = queryCombined.filter(queryCombined['id'] == docID1).collect()[0]
doc2 = queryCombined.filter(queryCombined['id'] == docID2).collect()[0]
top_similarity = cosine_similarity(doc1['abstractIDF'], doc2['abstractIDF'])
print('Abstract similarity: ', top_similarity)


Abstract similarity:  0.5515389226038091
